# Create Cloud Free Sentinel Mosaic
This notebook was designed to run in **Google Colab**.
It uses the Earth Engine Python API to generate cloud-free Sentinel-2 composites
over your area of interest, defined from a local shapefile or GeoJSON.

Getting Started:
- Upload your shapefile or GeoJSON to Colab (or mount Google Drive)
- Authenticate your Earth Engine account when prompted
- Run the notebook below to generate a mosaic and visualize it on an interactive map

⚠️ If you're not in Colab, you may need to adapt file paths and authentication flow.

In [1]:
import os
import geopandas as gpd
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import ee

In [2]:
# install geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) # geemap Python package is built upon ipyleaflet and folium packages

## 1. Authenticate and Initialize GEE

In [ ]:
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project = "ee-zmondo")

## 2. Load AOI from shapefile / GeoJSON

In [ ]:
# Get path to cwd and set project root
project_root = Path('/content')
bbox_path = project_root / 'gedi_waveforms_tf_bbox.geojson'

In [ ]:
# Load bounding box geojson as a GeoDataFrame
bbox_gdf = gpd.read_file(bbox_path).to_crs("EPSG:4326")

In [ ]:
from shapely.geometry import mapping

# Convert to an Earth Engine readable format (ee.FeatureCollection)
def gdf_to_ee(gdf):
  features = []
  for _, row in gdf.iterrows():
    geom_dict = mapping(row.geometry)
    attrs = row.drop('geometry').to_dict()
    feature = ee.Feature(geom_dict, attrs)
    features.append(feature)
  return ee.FeatureCollection(features)


## 3. Define spatial and temporal boundaries

In [8]:
AOI = gdf_to_ee(bbox_gdf)
START_DATE = '2021-03-15'
END_DATE = '2021-12-05'

## 4. Cloud masking Sentinel-2 with s2_cloudless and Mosaicking

In [ ]:
# Define masking parameters
CLOUD_FILTER = 100
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
SR_SCALE = 1e4

# Filter and join S2_SR_HARMONIZED with cloud probability
def get_s2_sr_cld_col(aoi, start_date, end_date):
    s2_sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))

    s2_cld = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY") \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date)

    # Join collections by system:index
    joined = ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr,
        'secondary': s2_cld,
        'condition': ee.Filter.equals(leftField='system:index', rightField='system:index')
    })

    return ee.ImageCollection(joined)

# Cloud detection
def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands([cld_prb.rename('cloud_probability'), is_cloud])

# Shadow detection
def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_SCALE).And(not_water).rename('dark_pixels')

    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    cld_proj = img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10) \
        .reproject(crs=img.select(0).projection(), scale=100) \
        .select('distance').mask().rename('cloud_transform')

    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    return img.addBands([dark_pixels, cld_proj, shadows])

# Combine cloud and shadow masks
def add_cld_shdw_mask(img):
    img = add_cloud_bands(img)
    img = add_shadow_bands(img)
    is_cld_shdw = img.select('clouds').add(img.select('shadows')).gt(0)

    mask = is_cld_shdw.focal_min(2).focal_max(BUFFER * 2 / 20) \
        .reproject(crs=img.select(0).projection(), scale=20) \
        .rename('cloudmask')

    return img.addBands(mask)

# Apply mask to reflectance bands
def apply_cld_shdw_mask(img):
    mask = img.select('cloudmask').Not()
    return img.select('B.*').updateMask(mask).copyProperties(img, img.propertyNames())

# Generate mosaic
def get_s2_mosaic(aoi, start_date, end_date):
    col = get_s2_sr_cld_col(aoi, start_date, end_date) \
        .map(add_cld_shdw_mask) \
        .map(apply_cld_shdw_mask)
    return col.median().clip(aoi)

In [ ]:
mosaic = get_s2_mosaic(AOI, START_DATE, END_DATE)

## 5. Visualize cloud-free Sentinel-2 mosaic

In [ ]:
# Initialize interactive map
Map = geemap.Map()

# Add basemap
Map.add_basemap('HYBRID')

# Define visualization parameters for RGB
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 3000
}

# Add the AOI
Map.addLayer(AOI, {}, 'AOI')

# Add sentinel-2 mosaic to map
Map.addLayer(mosaic, vis_params, "S2 Cloud-Free Mosaic", True)

# Zoom to AOI
Map.centerObject(AOI, 10)


Map

## 6. Processing Sentinel-1 Imagery, Applying Lee Speckle Filter, and Mosaicking

In [ ]:
# Speckle Filtering functions
# Credit: SERVIR-Mekong, adapted from
# https://mygeoblog.com/2021/01/21/sentinel-1-speckle-filter-refined-lee/
# Ported from JavaScript API to Python API with ChatGPT-4o

def db_to_power(img):
    """
    Converts a Sentinel-1 image from decibel (dB) scale to power scale.

    Parameters:
        img (ee.Image): Image in dB units.

    Returns:
        ee.Image: Image converted to power scale.
    """
    return ee.Image(10).pow(img.divide(10))

def power_to_db(img):
    """
    Converts a Sentinel-1 image from power scale to decibel (dB) scale.

    Parameters:
        img (ee.Image): Image in power units.

    Returns:
        ee.Image: Image converted to dB scale.
    """
    return ee.Image(10).multiply(img.log10())

def refined_lee_full(image):
    """
    Applies the full Refined Lee Speckle Filter to a multi-band Sentinel-1 image.

    Parameters:
        image (ee.Image): Multi-band SAR image in dB scale.

    Returns:
        ee.Image: Speckle-filtered image with original band names in dB scale.
    """
    image = db_to_power(image)
    band_names = image.bandNames()

    def filter_band(band_name):
        band_name = ee.String(band_name)
        img = image.select(band_name)

        # 3x3 mean and variance
        kernel3 = ee.Kernel.square(1)
        mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
        var3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

        # 7x7 sample kernel
        sample_kernel = ee.Kernel.fixed(7, 7, [
            [0,0,0,0,0,0,0],
            [0,1,0,1,0,1,0],
            [0,0,0,0,0,0,0],
            [0,1,0,1,0,1,0],
            [0,0,0,0,0,0,0],
            [0,1,0,1,0,1,0],
            [0,0,0,0,0,0,0]
        ], 3, 3, False)

        sample_mean = mean3.neighborhoodToBands(sample_kernel)
        sample_var = var3.neighborhoodToBands(sample_kernel)

        # Gradient directions
        gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs() \
            .addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs()) \
            .addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs()) \
            .addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())
        max_grad = gradients.reduce(ee.Reducer.max())
        gradmask = gradients.eq(max_grad)

        # Direction mask (collapsed to single band)
        directions = sample_mean.select(1).subtract(sample_mean.select(4)) \
            .gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1) \
            .add(sample_mean.select(6).subtract(sample_mean.select(4)) \
            .gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2)) \
            .add(sample_mean.select(3).subtract(sample_mean.select(4)) \
            .gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3)) \
            .add(sample_mean.select(0).subtract(sample_mean.select(4)) \
            .gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
        # Opposite directions (5-8)
        directions = directions \
            .add(directions.eq(0).multiply(5)) \
            .add(directions.eq(1).Not().multiply(6)) \
            .add(directions.eq(2).Not().multiply(7)) \
            .add(directions.eq(3).Not().multiply(8))

        # Reduce mask to single band before applying
        direction_mask = gradmask.reduce(ee.Reducer.anyNonZero())
        directions = directions.updateMask(direction_mask)

        # Noise estimate
        sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))
        sigmaV = (
            sample_stats.toArray()
            .arraySort()
            .arraySlice(0, 0, 5)
            .arrayReduce(ee.Reducer.mean(), [0])
            .arrayProject([0])
            .arrayFlatten([['sigmaV']])
            )

        # Directional kernels
        rect_weights = ee.List.repeat(ee.List.repeat(0, 7), 3).cat(ee.List.repeat(ee.List.repeat(1, 7), 4))
        diag_weights = ee.List([
            [1,0,0,0,0,0,0],
            [1,1,0,0,0,0,0],
            [1,1,1,0,0,0,0],
            [1,1,1,1,0,0,0],
            [1,1,1,1,1,0,0],
            [1,1,1,1,1,1,0],
            [1,1,1,1,1,1,1]
        ])
        rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
        diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

        dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
        dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

        for i in range(1, 4):
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

        dir_mean = dir_mean.reduce(ee.Reducer.sum())
        dir_var = dir_var.reduce(ee.Reducer.sum())

        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
        b = varX.divide(dir_var)
        result = dir_mean.add(b.multiply(img.subtract(dir_mean)))

        return power_to_db(result).rename(band_name)

    # Map over bands server-side using iterate
    def wrap_band(band_name, prev_result):
        prev_img = ee.Image(prev_result)
        return prev_img.addBands(filter_band(band_name))

    empty = ee.Image().select()
    filtered = band_names.iterate(wrap_band, empty)

    # Return and preserve metadata
    return ee.Image(filtered).copyProperties(image, image.propertyNames())


In [ ]:
def get_s1_mosaic(aoi, start_date, end_date,
                                   polarizations=['VV', 'VH'],
                                   orbit='ASCENDING',
                                   reducer='median'):
    """
    Creates a Sentinel-1 mosaic for the given time range and AOI,
    applying the full Refined Lee speckle filter to each image.

    Parameters:
        aoi (ee.Geometry): Area of interest.
        start_date (str): Start date (YYYY-MM-DD).
        end_date (str): End date (YYYY-MM-DD).
        polarizations (list): List of polarizations to include (e.g. ['VV', 'VH']).
        orbit (str): Orbit pass ('ASCENDING', 'DESCENDING', or 'BOTH').
        reducer (str): Reducer to apply over time ('median', 'mean', 'max', 'mosaic').

    Returns:
        ee.Image: A speckle-filtered S1 composite clipped to AOI.
    """
    s1 = (
        ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
        .filter(ee.Filter.eq('resolution_meters', 10))
        .select(polarizations)
    )

    if orbit in ['ASCENDING', 'DESCENDING']:
        s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit))

    # Apply full refined Lee filter
    s1_filtered = s1.map(refined_lee_full)

    # Reduce to a single mosaic
    if reducer == 'median':
        return s1_filtered.median().clip(aoi)
    elif reducer == 'mean':
        return s1_filtered.mean().clip(aoi)
    elif reducer == 'max':
        return s1_filtered.max().clip(aoi)
    elif reducer == 'mosaic':
        return s1_filtered.mosaic().clip(aoi)
    else:
        raise ValueError("Reducer must be one of: median, mean, max, mosaic")

In [ ]:
# Define temporal filters
S1_START_DATE = '2021-06-01'
S1_END_DATE = '2021-08-01'

In [ ]:
# Apply speckle filter function and mosaic
s1_mosaic = get_s1_mosaic(AOI, S1_START_DATE, S1_END_DATE)

## 7. Visualize Processed Sentinel-1 Mosaic

In [ ]:
# Add S1 VV and VH layers to existing map instance
Map.addLayer(s1_mosaic.select('VV'), {'min': -20, 'max': 0}, 'S1 VV, Lee Filtered')
Map.addLayer(s1_mosaic.select('VH'), {'min': -25, 'max': -5}, 'S1 VH, Lee Filtered')
Map

## 7. Export Sentinel Images to TFRecords for CNN

In [ ]:
# Create combined Sentinel-1 and Sentinel-2 image
sentinel_combined = mosaic.addBands(s1_mosaic)
sentinel_combined # inspect the combined sentinel image

In [ ]:
# Load GEDI point geometries from earth engine
gedi_fc = ee.FeatureCollection("projects/ee-zmondo/assets/encoded_latents_09_points")

In [ ]:
# Inspect the feature collection
gedi_fc.limit(1).getInfo()

## 8. Export combined Sentinel image to Google Cloud Storage Bucket

In [ ]:
import time

task = ee.batch.Export.image.toCloudStorage(
    image=sentinel_combined,
    description='gedi_pixelwise_5x5',
    bucket='ee-gedi-data',
    fileNamePrefix='tfrecords/gedi_5x5_patches',
    region=gedi_fc.geometry().bounds(),  # Large enough area
    scale=25,
    fileFormat='TFRecord',
    crs = 'EPSG:4326',
    formatOptions={
        'patchDimensions': [5, 5],
        'kernelSize': [5, 5],
        'compressed': True,
    }
)

task.start()